In [1]:
import pandas as pd
import numpy as np
import random

## Eddie's Code

In [2]:
glob_pop = []

# states
states = ['well', 'infected-a', 'infected-s', 'recovered', 'dead']
jobs = ['student', 'doctor', 'lawyer', 'police person', 'shop assistant', 'unemployed', 'retired']
job_type = ['student', 'high-income', 'low-income', 'retired']
class dude:
    def __init__(self, occupation, age, name, state=states[0], underlying_conditions=False, network=[]):
        self.job = occupation
        self.name = name
        self.age = age
        self.state = state
        self.under_cond = underlying_conditions
        self.network = network
        self.elapsed_infected = 0
        self.r_time = random.randint(4,14)

    def __str__(self):
        return "Job: {} Age: {} State: {}".format(self.job, self.age, self.state)

    def __repr__(self):
        return "Job: {} Age: {} State: {}".format(self.job, self.age, self.state)

    def infect(self):
        # The person is more likely to be symptomatic (but this is assumed).
        self.elapsed_infected = 0
        self.state = states[random.choices([1,2], [1,3])[0]]
        return self

    def uninfect(self):
        # The person was not actually infected
        self.state = states[0]
        return self

    def recover(self, val):
        # The person was infected but has now recovered
        self.state = states[val]
        return self

class group:
    def __init__(self, size, rate, code, type_):
        self.code = code
        self.type = type_
        self.transmission_rate = rate
        self.demographics = [0, 0, 0]
        self.infected_count = 0
        self.transmissions = 0
        self.size = size
        self.members = []
        
    def __str__(self):
        return "Size: {} Infected: {}".format(self.size, self.infected_count)

    def __repr__(self):
        return "Code: {} Size: {} Infected: {} T-rate: {:.2f}".format(self.code, self.size, self.infected_count, self.transmission_rate)

    def display(self):
        print("Code: {} Size: {} Infected: {}".format(self.code, self.size, self.infected_count))
        return

    def get_members(self):
        return self.members
    
    def plastic(self):
        self.infected_count += np.random.choice([1,0], p=[self.transmission_rate, 1-self.transmission_rate])
        
    def transmit(self):
        mems = random.sample(self.members, self.infected_count)
        for mem in mems:
            mem.infect()
        self.infected_count = 0

def get_group(g_list, code):
    for grp in g_list:
        if grp.code == code:
            return grp
        
def get_person(p_list, name):
    for p in p_list:
        if p.name == name:
            return p
    print('No person with this name')
    return None

def build_df(lst, what):
    if what == 'g':
        dct = {'code': [], 'type': [], 't-rate': [], 'size': []}
    
        for thing in lst:
            for tin in thing:
                dct['code'].append(tin.code)
                dct['type'].append(tin.type)
                dct['t-rate'].append(tin.transmission_rate)
                dct['size'].append(tin.size)
                
    else:
        dct = {'name': [], 'age': [], 'state': [], 'job': []}
    
        for thing in lst:
            for tin in thing:
                dct['name'].append(tin.name)
                dct['age'].append(tin.age)
                dct['state'].append(tin.state)
                dct['job'].append(tin.job)
    
    return pd.DataFrame(dct)

def create_houses(comm_size):
    houses = []
    
    for i in range(0, comm_size):
        rate = random.random()
        size = random.randint(1, 4)
        grp = group(size, rate, i, 'inner')
        houses.append(grp)
    return houses

def init_peeps(houses):
    peeps = []
    
    for i in range(0, len(houses)):
        job = random.choice(job_type)
        age = random.randint(18, 80)
        person = dude(job, age, i, network=[houses[i]])
        houses[i].members.append(person)
        peeps.append(person)
        
    return peeps

def populate_houses(people, hs):
    pep_cnt = len(people)
    
    for house in hs:
        job = house.members[0].job

        for new_member in range(0, house.size - 1):
            age = abs(house.members[0].age + random.randint(-5, 5))
            person = dude(job, age, pep_cnt, network=[house])
            house.members.append(person)
            people.append(person)
            pep_cnt +=1
            
    return

def genesis(comm_size):
    houses = create_houses(comm_size)
    
    peeps = init_peeps(houses)
    
    populate_houses(peeps, houses)
    
    return houses, peeps


### Initial population creation and first circle

In [3]:
houses, peeps = genesis(30000)

### Second Circle

In [4]:
len(peeps)

74843

In [5]:
# Size of outer circles, dependent on populaiton size
ocircle_size = int(len(peeps) * 0.008 + random.randint(-5, 5))

In [6]:
# Number of outer circles, dependent on inner circle count
ocircle_count = int((len(peeps) / ocircle_size) * 0.95)

In [7]:
ocircle_size * ocircle_count

70924

In [8]:
ocircs = []
for i in range(0, ocircle_count):
    # Lower rates for outr circles
    rate = random.uniform(0,0.5)
    size = int(len(peeps) * 0.008 + random.randint(-5, 5))
    grp = group(size, rate, i, 'outer')
    ocircs.append(grp)

In [9]:
for pepe in peeps:
    nets = random.sample(ocircs, random.randint(1,3))
    for net in nets:
        net.members.append(pepe)
        pepe.network.append(net)

In [10]:
networks = houses + ocircs

### Explore the Networks

In [24]:
pepsam = random.sample(peeps, 5)

for pep in pepsam:
    print(pep.name, pep)
    for net in pep.network:
        print(net.code, net)

30285 Job: high-income Age: 47 State: 0
188 Size: 4 Infected: 0
57 Size: 599 Infected: 0
63 Size: 603 Infected: 0
66907 Job: retired Age: 79 State: 0
24639 Size: 3 Infected: 0
87 Size: 602 Infected: 0
94 Size: 595 Infected: 0
99 Size: 600 Infected: 0
1533 Job: student Age: 37 State: 0
1533 Size: 4 Infected: 0
116 Size: 602 Infected: 0
69905 Job: retired Age: 69 State: 0
26620 Size: 4 Infected: 0
65 Size: 599 Infected: 0
59 Size: 598 Infected: 0
72 Size: 604 Infected: 0
39229 Job: low-income Age: 73 State: 0
6193 Size: 3 Infected: 0
86 Size: 600 Infected: 0


In [25]:
get_group(ocircs, 32)

Code: 32 Size: 594 Infected: 0 T-rate: 0.26

In [26]:
for cir in ocircs:
    if cir.code == 30:
        cir.display()

Code: 30 Size: 599 Infected: 0


In [27]:
print(random.choice(random.choice(ocircs).members))

Job: retired Age: 59 State: 0


In [28]:
print(peeps[5].network[0])

Size: 3 Infected: 0


### Community Code Infrastructure

In [13]:
build_df([houses, ocircs], 'g')

,code,type,t-rate,size
0,0,inner,0.000849,2
1,1,inner,0.257237,3
2,2,inner,0.897786,3
3,3,inner,0.152369,1
4,4,inner,0.070122,3
...,...,...,...,...
30113,113,outer,0.274119,600
30114,114,outer,0.248518,596
30115,115,outer,0.418092,599
30116,116,outer,0.214742,600


In [12]:
circs = build_df([networks], 'g')

In [13]:
circs[circs['type'] == 'outer'].sort_values('t-rate', ascending=False)

,code,type,t-rate,size
30012,12,outer,0.499209,594
30026,26,outer,0.497126,594
30000,0,outer,0.496561,598
30070,70,outer,0.493727,599
30050,50,outer,0.492916,595
...,...,...,...,...
30073,73,outer,0.009176,603
30105,105,outer,0.005899,599
30100,100,outer,0.004134,601
30080,80,outer,0.003362,594


In [14]:
build_df([peeps], 'p').tail()

,name,age,state,job
75055,75055,47,well,high-income
75056,75056,44,well,high-income
75057,75057,40,well,low-income
75058,75058,41,well,low-income
75059,75059,38,well,low-income


### Transmission Simulation

In [11]:
# Begin by infecting one person
r_pep = random.choice(peeps)
print(r_pep.name, r_pep.state)
r_pep.infect()
print(r_pep.name, r_pep.state)

18926 well
18926 infected-s


In [12]:
get_group(ocircs, 82).infected_count

0

In [21]:
get_person(peeps, 18926).r_time

5

In [26]:
get_person(peeps, 18926).elapsed_infected

3

In [64]:
for person in peeps:
    if (person.state == states[1]) or (person.state == states[2]):
        person.elapsed_infected += 1
        for net in person.network:
            net.plastic()
        if person.elapsed_infected >= person.r_time:
            person.recover(np.random.choice([3,4], p=[0.95, 0.05]))
            

In [65]:
for n in networks:
    if n.infected_count > 0:
        n.transmit()
        

In [66]:
build_df([peeps], 'p')['state'].value_counts()

well          63105
infected-s     8746
infected-a     2934
recovered        56
dead              2
Name: state, dtype: int64